### Random Model



In [2]:
import argparse
import sys
import numpy as np
import matplotlib.pyplot as plt

import gym
from gym import wrappers, logger


EPISODES = 500
ALL_SCORES = np.zeros(EPISODES)

env = gym.make("MsPacman-ram-v0")
env = wrappers.Monitor(env, '/tmp/MsPacman-ram-experiment-1',force=True)

for episode in range(EPISODES):
    env.reset()
    
    reward, info, done = None, None, None

    
    total_score = 0
    while done != True:
        env.render()
        random_action = env.action_space.sample()
        state, reward, done, info = env.step(random_action)
        total_score += reward
    ALL_SCORES[episode] = total_score
    print("Total Score: {}".format(total_score))
    # print(state, reward, done, info)
    
env.close()
plt.plot(ALL_SCORES)
plt.title("Random Agent: {} Episodes".format(EPISODES))
plt.show()

print("-------------------------")
print('Average Score for {} Episodes: {}'.format(EPISODES, np.mean(ALL_SCORES)))


Total Score: 240.0
Total Score: 230.0
Total Score: 150.0
Total Score: 210.0
Total Score: 340.0
Total Score: 220.0
Total Score: 140.0
Total Score: 240.0
Total Score: 180.0
Total Score: 200.0
Total Score: 130.0
Total Score: 230.0
Total Score: 190.0
Total Score: 150.0
Total Score: 250.0
Total Score: 210.0
Total Score: 280.0
Total Score: 250.0
Total Score: 1010.0
Total Score: 110.0
Total Score: 180.0
Total Score: 200.0
Total Score: 220.0
Total Score: 160.0
Total Score: 270.0
Total Score: 280.0
Total Score: 140.0
Total Score: 260.0
Total Score: 210.0
Total Score: 130.0
Total Score: 180.0
Total Score: 210.0
Total Score: 150.0
Total Score: 150.0
Total Score: 220.0
Total Score: 190.0
Total Score: 390.0
Total Score: 110.0
Total Score: 270.0
Total Score: 240.0
Total Score: 200.0
Total Score: 330.0
Total Score: 190.0
Total Score: 320.0
Total Score: 280.0
Total Score: 190.0
Total Score: 210.0
Total Score: 200.0
Total Score: 130.0
Total Score: 230.0
Total Score: 170.0
Total Score: 350.0
Total Score

KeyboardInterrupt: 

v2- corrected reward (0.01 LR, batch size 256, min epsilon 0.05, episilon decay 0.9999)  
v2- corrected reward (0.05 LR, batch size 256, min epsilon 0.0, episilon decay 0.99999), increased memeory from 2000 to 10000

BOTH STILL GET STUCK

## Training Model

In [1]:
import sys
import gym
import pylab
import random
import numpy as np
from collections import deque
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import Sequential
from gym import wrappers


# DQN Agent for the MsPacman
# it uses Neural Network to approximate q function and replay memory & target q network
class DQNAgent:
    def __init__(self, state_size, action_size):
        # if you want to see MsPacman learning, then change to True
        self.render = True
        self.load_model = False

        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # These are hyper parameters for the DQN
        self.discount_factor = 0.99
        self.learning_rate = 0.05
        self.epsilon = 1.0
        self.epsilon_decay = 0.99999
        self.epsilon_min = 0.0
        self.batch_size = 256
        self.train_start = 1000
        
        # create replay memory using deque
        self.memory = deque(maxlen=10000)

        # create main model
        self.model = self.build_model()

        if self.load_model:
            self.model.load_weights("./Saved Weights/pacman.h5")

    # approximate Q function using Neural Network
    # state is input and Q Value of each action is output of network
    def build_model(self):
        model = Sequential()
        model.add(Dense(128, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(self.action_size, activation='linear', kernel_initializer='he_uniform'))
        model.summary()
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    # get action from model using epsilon-greedy policy
    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model.predict(state)
            return np.argmax(q_value[0])

    # save sample <s,a,r,s'> to the replay memory
    def append_sample(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    # pick samples randomly from replay memory (with batch_size)
    def train_model(self):

        if len(self.memory) < self.train_start:
            return
        batch_size = min(self.batch_size, len(self.memory))
#         total_training_steps += batch_size

        mini_batch = random.sample(self.memory, batch_size)

        update_input = np.zeros((batch_size, self.state_size))
        update_target = np.zeros((batch_size, self.state_size))
        action, reward, done = [], [], []

        for i in range(self.batch_size):
            update_input[i] = mini_batch[i][0] #STATE
            action.append(mini_batch[i][1])    #ACTION
            reward.append(mini_batch[i][2])    #REWARD
            update_target[i] = mini_batch[i][3]#NEXT STATE
            done.append(mini_batch[i][4])      #DONE

        target = self.model.predict(update_input)
        target_val = self.model.predict(update_target)


        for i in range(self.batch_size):
            # Q Learning: get maximum Q value at s' from model
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                target[i][action[i]] = reward[i] + self.discount_factor * (np.amax(target_val[i]))
#         print(update_input, update_target)

        # and do the model fit!
        self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=0)
        


if __name__ == "__main__":
    EPISODES = 1000
    env = gym.make('MsPacman-ram-v0')
    env = wrappers.Monitor(env, '/tmp/MsPacman-ram-experiment-1',force=True)
    
    # get size of state and action from environment
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    agent = DQNAgent(state_size, action_size)

    scores, episodes = [], []

    for e in range(EPISODES):
        done = False
        score = 0
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        lives = 3
        while not done: 
            dead = False         
            while not dead:
                if agent.render:
                    env.render()

                # get action for the current state and go one step in environment
                action = agent.get_action(state)
                next_state, reward, done, info = env.step(action)
                reward = reward-1 if not dead else -1000  # if action make Pacman dead, then gives penalty of -100
                next_state = np.reshape(next_state, [1, state_size])
                
               
                
                
                # save the sample <s, a, r, s'> to the replay memory
                agent.append_sample(state, action, reward, next_state, done)
                
                # every time step do the training
                agent.train_model()

                state = next_state            
                score += reward
                dead = info['ale.lives'] < lives
                lives = info['ale.lives']
                


            if done:
                scores.append(score)
                episodes.append(e)
                pylab.plot(episodes, scores, 'b')
                pylab.savefig("./pacman.png")
                print("episode:", e, "  score:", score, "  memory length:",
                      len(agent.memory), "  epsilon:", agent.epsilon)
    #         # save the model
        if e % 2 == 0:
            agent.model.save_weights("./Saved Weights/pacman_correctedv12.h5")

#     print("Total Training Steps: {}".format(total_training_steps))


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 585       
Total params: 25,353
Trainable params: 25,353
Non-trainable params: 0
_________________________________________________________________
episode: 0   score: -488.0   memory length: 748   epsilon: 0.9925478684572383
episode: 1   score: -507.0   memory length: 1435   epsilon: 0.9857523997856545
episode: 2   score: -324.0   memory length: 1889   epsilon: 0.9812872052258903
episode: 3   score: -324.0   memory length: 2483   epsilon: 0.9754756077014011
episode: 4   score: -384.0   memory length: 3117   epsilon: 0.9693106251687865
episode:

episode: 93   score: -397.0   memory length: 10000   epsilon: 0.5300928105516604
episode: 94   score: -545.0   memory length: 10000   epsilon: 0.5245297005889874
episode: 95   score: -386.0   memory length: 10000   epsilon: 0.5209437228795372
episode: 96   score: -277.0   memory length: 10000   epsilon: 0.5162917298063356
episode: 97   score: -400.0   memory length: 10000   epsilon: 0.5125365046508248
episode: 98   score: -315.0   memory length: 10000   epsilon: 0.5092412662875729
episode: 99   score: -254.0   memory length: 10000   epsilon: 0.5066811344563399
episode: 100   score: -379.0   memory length: 10000   epsilon: 0.5033530671660001
episode: 101   score: -394.0   memory length: 10000   epsilon: 0.5000218579977569
episode: 102   score: -333.0   memory length: 10000   epsilon: 0.49696608424481997
episode: 103   score: -378.0   memory length: 10000   epsilon: 0.4942501449937592
episode: 104   score: -341.0   memory length: 10000   epsilon: 0.49143600461199966
episode: 105   score:

episode: 193   score: -85.0   memory length: 10000   epsilon: 0.2595887208491703
episode: 194   score: -285.0   memory length: 10000   epsilon: 0.2575589166464798
episode: 195   score: -298.0   memory length: 10000   epsilon: 0.2558697340344737
episode: 196   score: -201.0   memory length: 10000   epsilon: 0.254947704948297
episode: 197   score: -341.0   memory length: 10000   epsilon: 0.25235791928521095
episode: 198   score: -223.0   memory length: 10000   epsilon: 0.2511670789304479
episode: 199   score: -186.0   memory length: 10000   epsilon: 0.24957471742378298
episode: 200   score: -297.0   memory length: 10000   epsilon: 0.2465064790071109
episode: 201   score: -284.0   memory length: 10000   epsilon: 0.24497306015348885
episode: 202   score: -157.0   memory length: 10000   epsilon: 0.24273691440902537
episode: 203   score: -154.0   memory length: 10000   epsilon: 0.2411545869279244
episode: 204   score: -200.0   memory length: 10000   epsilon: 0.23932874990715658
episode: 205 

episode: 293   score: -186.0   memory length: 10000   epsilon: 0.12099203522232775
episode: 294   score: -302.0   memory length: 10000   epsilon: 0.12007357509887093
episode: 295   score: -204.0   memory length: 10000   epsilon: 0.11927892374910348
episode: 296   score: -158.0   memory length: 10000   epsilon: 0.11876949835632727
episode: 297   score: -314.0   memory length: 10000   epsilon: 0.11746548795643603
episode: 298   score: -71.0   memory length: 10000   epsilon: 0.11672661064257853
episode: 299   score: -214.0   memory length: 10000   epsilon: 0.11564145424249615
episode: 300   score: -355.0   memory length: 10000   epsilon: 0.11497842061391718
episode: 301   score: -408.0   memory length: 10000   epsilon: 0.1139505313874562
episode: 302   score: -278.0   memory length: 10000   epsilon: 0.11304482021237
episode: 303   score: -318.0   memory length: 10000   epsilon: 0.11232588650193771
episode: 304   score: -179.0   memory length: 10000   epsilon: 0.11176677367938816
episode: 

episode: 393   score: -8.0   memory length: 10000   epsilon: 0.05713637575886294
episode: 394   score: -285.0   memory length: 10000   epsilon: 0.05648024379443767
episode: 395   score: -294.0   memory length: 10000   epsilon: 0.05594957633092786
episode: 396   score: -217.0   memory length: 10000   epsilon: 0.05567776293795685
episode: 397   score: -75.0   memory length: 10000   epsilon: 0.05528108476396874
episode: 398   score: -301.0   memory length: 10000   epsilon: 0.05485101893473145
episode: 399   score: 147.0   memory length: 10000   epsilon: 0.05455398359751995
episode: 400   score: -180.0   memory length: 10000   epsilon: 0.0542873216000236
episode: 401   score: -34.0   memory length: 10000   epsilon: 0.05393883284469524
episode: 402   score: -319.0   memory length: 10000   epsilon: 0.05365961410552291
episode: 403   score: -266.0   memory length: 10000   epsilon: 0.05330342671523259
episode: 404   score: 32.0   memory length: 10000   epsilon: 0.052821619904823425
episode: 40

episode: 492   score: -308.0   memory length: 10000   epsilon: 0.027042973918716854
episode: 493   score: -103.0   memory length: 10000   epsilon: 0.026923437914317276
episode: 494   score: -153.0   memory length: 10000   epsilon: 0.026758901237367138
episode: 495   score: -229.0   memory length: 10000   epsilon: 0.02650615915891348
episode: 496   score: -362.0   memory length: 10000   epsilon: 0.026349705474260497
episode: 497   score: -367.0   memory length: 10000   epsilon: 0.026192865581287913
episode: 498   score: -521.0   memory length: 10000   epsilon: 0.02592679569149883
episode: 499   score: -315.0   memory length: 10000   epsilon: 0.02569578502976694
episode: 500   score: 130.0   memory length: 10000   epsilon: 0.025440106424762462
episode: 501   score: -446.0   memory length: 10000   epsilon: 0.025208137927745147
episode: 502   score: -337.0   memory length: 10000   epsilon: 0.025080655472141885
episode: 503   score: -310.0   memory length: 10000   epsilon: 0.024953069113062

episode: 591   score: -236.0   memory length: 10000   epsilon: 0.013114042578158375
episode: 592   score: 24.0   memory length: 10000   epsilon: 0.013007466878067607
episode: 593   score: -168.0   memory length: 10000   epsilon: 0.012916990436994397
episode: 594   score: -229.0   memory length: 10000   epsilon: 0.012859122870499468
episode: 595   score: -124.0   memory length: 10000   epsilon: 0.012762529216788922
episode: 596   score: -287.0   memory length: 10000   epsilon: 0.012692908420340975
episode: 597   score: -270.0   memory length: 10000   epsilon: 0.012594288359803455
episode: 598   score: 21.0   memory length: 10000   epsilon: 0.012494060438429998
episode: 599   score: -164.0   memory length: 10000   epsilon: 0.01240020901795737
episode: 600   score: -445.0   memory length: 10000   epsilon: 0.012266393343829429
episode: 601   score: -165.0   memory length: 10000   epsilon: 0.012159530076361795
episode: 602   score: -400.0   memory length: 10000   epsilon: 0.0120867909677519

KeyboardInterrupt: 

### FINDING

The Pacman learns to just stay in the corner after consuming the "Power Pellets" since the reward from consuming ghosts outweight the negative reward of dying. Must increase the negative reward from dying.

The learned weights will be in 'correctedv1.h5'

# Code to Test Trained Weights- No Training Involved

In [1]:
# DQN Agent for the MsPacman
# it uses Neural Network to approximate q function and replay memory & target q network
class TEST_DQNAgent:
    def __init__(self, state_size, action_size):
        # if you want to see MsPacman learning, then change to True
        self.render = True
        self.load_model = True
        self.epsilon = 0.1

        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        
        # create main model
        self.model = self.build_model()

        if self.load_model:
            self.model.load_weights("./Saved Weights/pacman_correctedv12.h5")

    # approximate Q function using Neural Network
    # state is input and Q Value of each action is output of network
    def build_model(self):
        model = Sequential()
        model.add(Dense(128, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(self.action_size, activation='linear', kernel_initializer='he_uniform'))
        model.summary()
        model.compile(loss='mse', optimizer=Adam())
        return model

    # get action from model using epsilon-greedy policy
    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model.predict(state)
            return np.argmax(q_value[0])
#         q_value = self.model.predict(state)
#         return np.argmax(q_value[0])
        
        
import sys
import gym
import pylab
import random
import numpy as np
from collections import deque
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import Sequential
from gym import wrappers
import matplotlib.pyplot as plt


if __name__ == "__main__":
    EPISODES = 100
    ALL_SCORES = np.zeros(EPISODES)

    env = gym.make('MsPacman-ram-v0')
    env = wrappers.Monitor(env, '/tmp/MsPacman-ram-experiment-1',force=True)
    # get size of state and action from environment
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    agent = TEST_DQNAgent(state_size, action_size)

    scores, episodes = [], []

    for e in range(EPISODES):
        done = False
        score = 0
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        lives = 3
        while not done: 
            dead = False         
            while not dead:
                if agent.render:
                    env.render()

                # get action for the current state and go one step in environment
                action = agent.get_action(state)
                next_state, reward, done, info = env.step(action)
                next_state = np.reshape(next_state, [1, state_size])

                state = next_state            
                score += reward
                dead = info['ale.lives']<lives
                lives = info['ale.lives']
                # if an action make the Pacman dead, then gives penalty of -100
                reward = reward if not dead else -500

            if done:
                scores.append(score)
                episodes.append(e)
                pylab.plot(episodes, scores, 'b')
                pylab.savefig("./pacman.png")
                print("episode:", e, "  score:", score)
        
        ALL_SCORES[e] = score
                
    env.close()
    plt.plot(ALL_SCORES)
    plt.title("Random Agent: {} Episodes".format(EPISODES))
    plt.show()

    print("-------------------------")
    print('Average Score for {} Episodes: {}'.format(EPISODES, np.mean(ALL_SCORES)))

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 585       
Total params: 25,353
Trainable params: 25,353
Non-trainable params: 0
_________________________________________________________________
episode: 0   score: 470.0
episode: 1   score: 420.0
episode: 2   score: 550.0
episode: 3   score: 300.0
episode: 4   score: 180.0
episode: 5   score: 390.0
episode: 6   score: 260.0
episode: 7   score: 180.0
episode: 8   score: 180.0
episode: 9   score: 320.0
episode: 10   score: 320.0
episode: 11   score: 160.0
episode: 12   score: 240.0
episode: 13   score: 300.0
episode: 14   score: 160.0
episode

KeyboardInterrupt: 